In [ ]:
import ipywidgets.widgets as widgets
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
import cv2
import io
from jetcam.csi_camera import CSICamera
import time


In [ ]:
def preprocess_image(image, target_size=(224, 224)):
    image = cv2.resize(image, target_size)
    image = image / 255.0
    return image.transpose(2, 0, 1)  # CHW format

yellow_ref_hue = 40
crop_top_percent = 0.40

# Define HSV range for orange
orange_lower = np.array([24, 60, 60])  # Lower bound for hue, saturation, value
orange_upper = np.array([172, 255, 255])  # Upper bound for hue, saturation, value


def compute_yellow_score(img: np.ndarray) -> np.ndarray:
    """
    Convert RGB image to yellow detection score (0-1 scale).
    Args: img: RGB image array of shape (H, W, 3)
    Returns: Array of shape (H, W) with values 0-1 indicating yellow intensity
    """
    # Convert RGB to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    
    h = h.astype(np.int16) 
    h_diff = np.abs(h - yellow_ref_hue)
    hue_diff = np.minimum(h_diff, 180 - h_diff)
    hue_score = 1 - (hue_diff / 90.0)

    s_mask = s > 100
    v_mask = (v > 50) & (v < 200)
    
    combined_mask = s_mask & v_mask
    
    # Weighted average
    yellow_score = hue_score * (7/16) + (s / 255.0) * (5/16) + (v / 255.0) * (4/16) 
    yellow_score *= yellow_score
    yellow_score /= np.max(yellow_score) if np.max(yellow_score) > 0 else 1

    yellow_score_uint8 = (yellow_score * 255).astype(np.uint8)
    threshold, _ = cv2.threshold(yellow_score_uint8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return (yellow_score_uint8 >= threshold).astype(np.uint8)

def detect_orange(image: np.ndarray) -> np.ndarray:
    """
    Detect orange areas in an image and return a binary mask highlighting those areas.
    Args:
        image: RGB image array of shape (H, W, 3)
    Returns:
        Binary mask of shape (H, W) where orange areas are white (255) and others are black (0)
    """
    # Convert image to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Apply the color range mask for orange
    orange_mask = cv2.inRange(hsv_image, orange_lower, orange_upper)
    
    # Optionally apply some morphological operations to clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    orange_mask = cv2.morphologyEx(orange_mask, cv2.MORPH_OPEN, kernel)
    orange_mask = cv2.morphologyEx(orange_mask, cv2.MORPH_CLOSE, kernel)

    return orange_mask

def preprocess(obs: np.ndarray, target_size=(224, 224)) -> np.ndarray:
    """
    Preprocess the observation by resizing and applying the orange detection mask.
    """
    if obs is None:
        return None
    obs = cv2.resize(obs, target_size)

    # Crop top portion if needed
    crop_height = 135
    cropped = obs[-crop_height:, :, :]

    # Detect orange areas in the cropped image
    orange_mask = detect_orange(cropped)
    
    return orange_mask[..., np.newaxis]  # Add a channel dimension




In [ ]:
print("Initializing video capture...")
camera = CSICamera(width=224, height=224, capture_fps=65)
print("Camera ready.")


In [ ]:
try:
    while True:
        # Capture an image from the camera
        image = camera.read()

        # Preprocess the image
        orange_mask = preprocess(image)

        display_image = cv2.cvtColor(orange_mask, cv2.COLOR_GRAY2RGB)
        pil_img = Image.fromarray(display_image)

        # Display the image in the notebook
        buf = io.BytesIO()
        pil_img.save(buf, format='PNG')
        buf.seek(0)

        clear_output(wait=True)
        display(Image.open(buf))
except KeyboardInterrupt:
    print("Closed.")
finally:
    print("Exiting...")
        
        
        

In [ ]:
import cv2
import numpy as np
from IPython.display import display, clear_output
import io
from PIL import Image

def display_crosshair_and_rgb(camera):
    """
    Display the camera feed with a crosshair in the center and print the RGB values at the crosshair.
    """
    while True:
        # Capture an image from the camera
        image = camera.read()

        # Get the image dimensions and calculate the center
        height, width, _ = image.shape
        center_x, center_y = width // 2, height // 2

        # Convert the image to HSV
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Get the HSV values at the center before drawing the crosshair
        hsv_value = hsv_image[center_y, center_x].tolist()
        print(f"HSV value at crosshair: {hsv_value}")

        # Draw a crosshair on the image after obtaining the HSV value
        crosshair_color = (0, 255, 0)  # Green color for the crosshair
        cv2.drawMarker(image, (center_x, center_y), crosshair_color, markerType=cv2.MARKER_CROSS, 
                       markerSize=20, thickness=2, line_type=cv2.LINE_AA)

        # Convert BGR to RGB for display
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_image)

        # Display the image in the notebook
        buf = io.BytesIO()
        pil_img.save(buf, format='JPEG')
        buf.seek(0)
        
        clear_output(wait=True)
        display(Image.open(buf))

# Call the function to start displaying the camera feed with the crosshair
display_crosshair_and_rgb(camera)
